In [27]:
import requests

from bs4 import BeautifulSoup

import pandas as pd


### Finding and extracting contents of table in Wiki page


In [28]:
URL = r"https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

res = requests.get(URL).text
soup = BeautifulSoup(res,'lxml')


In [29]:
data=pd.read_html(URL, attrs={"class": "wikitable"})
data=data[0]     

In [30]:
data

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


#### Checking where *Not assigned* appear in *Borough*

In [31]:
data[data['Borough'] =="Not assigned"]

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
7,M8A,Not assigned,Not assigned
10,M2B,Not assigned,Not assigned
15,M7B,Not assigned,Not assigned
...,...,...,...
174,M4Z,Not assigned,Not assigned
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned


#### Removing rows in which the *Borough* field is "Not assigned" as it will not return any result in geocoder

In [32]:
data.drop(data[data['Borough'] =="Not assigned"].index,axis=0,inplace=True)
data.reset_index(drop=True,inplace=True)

In [35]:
data.shape

(103, 3)